#Assignment Advance Data Science
###Topic Name -  Worked Examples Granger Causality and Causal Interference
Name -  Viral Jangale

Nuid - 002654463

####Here, I am using different datasets for Granger Causality and Causal Inference
####Instead of accomodity prices dataset, I am using creditcard dataset on my granger causality.
####For Causal Inference I have used water quality dataset instead of hotel bookings

This command will install the `fredapi` package from the Python Package Index (PyPI) into your Python environment, allowing you to use the functionalities provided by this package to interact with the Federal Reserve Economic Data (FRED) API.

In [ ]:
# !pip install fredapi

In [ ]:
import numpy as np
import pandas as pd
import os

from sklearn.metrics import mean_squared_error as mse

import matplotlib.pyplot as plt
import matplotlib.dates as md
%matplotlib inline

# from fredapi import Fred
# fred = Fred(api_key='fred')

from statsmodels.tsa.stattools import adfuller

from statsmodels.tsa.api import VAR
from statsmodels.tsa.ar_model import AutoReg

plt.style.use('fivethirtyeight')

In [ ]:
def my_rmse(x,y):
    return(np.round( np.sqrt(mse(x,y)) ,4))

In [ ]:
# general settings
class CFG:
    img_dim1 = 20
    img_dim2 = 10
    tail = 30
    varlag = 4

# adjust the parameters for displayed figures
plt.rcParams.update({'figure.figsize': (CFG.img_dim1,CFG.img_dim2)})


<a id="section-granger"></a>
# Granger causality

Granger causality is a statistical concept used to determine whether one time series can predict another. Named after economist Clive Granger, this concept is particularly useful in analyzing cause-and-effect relationships in time-series data without necessarily implying direct causation in the traditional sense.

Here's a breakdown of Granger causality:

### Time Series Data:
- **Time Series:** A sequence of data points recorded in chronological order.

### Granger Causality Principle:
- **Basic Idea:** If a variable (let's say Variable A) helps predict another variable (Variable B) better than just the past values of Variable B alone, then Variable A is said to have Granger-causal influence on Variable B.
- **Time Lag:** Granger causality examines whether past values of one variable help predict future values of another variable.
- **Statistical Test:** It involves running statistical tests (often F-tests) to compare predictive models that include and exclude past values of the potential causal variable.
  
<img src="https://warwick.ac.uk/fac/sci/dcs/research/combi/research/compneuro/granger.png?maxWidth=600&maxHeight=373">

### Application:
- **Economics and Finance:** Commonly used in economics and finance to explore relationships between economic indicators, like GDP, interest rates, and consumer spending.
- **Science and Engineering:** Applied in various fields like neuroscience to understand brain activity, in meteorology for weather prediction, and in engineering for system analysis.

### Considerations:
- **Not True Causality:** Granger causality doesn't prove direct causation; it identifies predictive relationships between variables.
- **External Factors:** It doesn't account for all possible factors or external influences that might affect both variables being studied.

### Interpretation:
- **Positive Result:** If Granger causality is found, it suggests that one time series provides valuable information for predicting the future of another series.
- **Negative Result:** A lack of Granger causality doesn't necessarily mean there's no relationship between the variables; it just means the tested variable might not significantly improve predictions of the other variable.

### Key Steps:
1. **Data Preparation:** Organize the time series data for analysis.
2. **Model Specification:** Select appropriate lags and variables for the Granger causality test.
3. **Statistical Testing:** Use appropriate statistical tests to determine the presence of Granger causality.
4. **Interpretation:** Analyze the results and draw conclusions about the predictive relationship between the variables.

Granger causality is a valuable tool for understanding predictive relationships between time series data and identifying potential lead-lag associations but should be used cautiously and in conjunction with domain knowledge to draw meaningful conclusions about causation.
* Establishing causality in observational data is notoriously difficult

* Granger causality is a lower bar

* Original idea: comparing the predictive power for a process $Y$ using all the available information $U$ with the same using $U\backslash X$

* discarding X reduces the predictive power regarding Y $\implies$ X contains some unique information regarding Y $\rightarrow$ **X Granger-causes Y**

* in practice: if past (lagged) values of $X$ improve prediction of $Y$, conditional on its own past values, then $X$ Granger-causes $Y$

* Granger’s causality test: $H_0$ = coefficients of past values in the regression equation is zero $\rightarrow$ null is no causality


We will demonstrate Granger causality usefulness by predicting the price of evolution of major precious metals:
<img src="https://www.wallstreetmojo.com/wp-content/uploads/2023/10/Granger-Causality.png">

In [ ]:
df = pd.read_csv('https://github.com/Viral-28/ADS/raw/main/BankChurners.csv',
                  usecols = ['date','Credit_Limit', 'Contacts_Count_12_mon', 'Customer_Age'])
# process timestamp
#df['date'] = pd.to_datetime(df['date'].apply(lambda s: s.replace('M', '-')))

df = df.set_index('date').rename_axis('metal', axis=1)

df.head()

metal,Customer_Age,Contacts_Count_12_mon,Credit_Limit
date,,,
1960M01,45,3,12691.0
1960M02,49,2,8256.0
1960M03,51,0,3418.0
1960M04,40,1,3313.0
1960M05,40,0,4716.0


In [ ]:
import plotly.express as px

fig = px.line(df, facet_col="metal", facet_col_wrap=1)
fig.update_yaxes(matches=None)
fig.show()

In [ ]:
# pd.plotting.lag_plot(df)

The visual representation above strongly indicates that the series lack stationarity, a necessary condition for constructing a VAR model. To confirm this notion, we'll put it to the test using the Augmented Dickey-Fuller method, a commonly employed approach.

Null hypothesis ($H_0$): The time series does not exhibit stationarity and instead maintains stationarity around a trend.

Alternative hypothesis ($H_1$): The time series demonstrates stationarity.

In [ ]:
def adf_test(df):
    result = adfuller(df.values)
    print('ADF Statistics: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

for f in df.columns:
    print('ADF Test: ' + str(f))
    adf_test(df[f])
    print('---')

ADF Test: Customer_Age
ADF Statistics: -11.331363
p-value: 0.000000
Critical values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
---
ADF Test: Contacts_Count_12_mon
ADF Statistics: -7.064604
p-value: 0.000000
Critical values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
---
ADF Test: Credit_Limit
ADF Statistics: -5.750433
p-value: 0.000001
Critical values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
---


All p-values above any reasonable critical level $\implies$ series are not stationary

An alternative method, the KPSS test, assesses whether the series displays stationarity around a mean or linear trend or if it's non-stationary due to a unit root.

Null hypothesis ($H_0$): The time series exhibits stationarity.

Alternative hypothesis ($H_1$): The time series lacks stationarity.

In [ ]:
from statsmodels.tsa.stattools import kpss

def kpss_test(df):
    statistic, p_value, n_lags, critical_values = kpss(df.values)

    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')

for f in ['Credit_Limit', 'Contacts_Count_12_mon', 'Customer_Age']:
    print('KPSS Test: ' + str(f))
    kpss_test(df[f])
    print('---')

KPSS Test: Credit_Limit
KPSS Statistic: 2.4986647020231105
p-value: 0.01
num lags: 45
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
---
KPSS Test: Contacts_Count_12_mon
KPSS Statistic: 5.07303473703903
p-value: 0.01
num lags: 42
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
---
KPSS Test: Customer_Age
KPSS Statistic: 0.952302929989896
p-value: 0.01
num lags: 28
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
---


<ipython-input-12-7c4b2269b00f>:4: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.


<ipython-input-12-7c4b2269b00f>:4: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.


<ipython-input-12-7c4b2269b00f>:4: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.




The KPSS test examines whether a set of numbers, like the prices of gold, platinum, or silver, behaves consistently over time. Its goal is to determine if these numbers tend to stay around an average value (stationary) or if they frequently wander away from it (non-stationary). The test presents two possibilities: the null hypothesis suggests stability around an average, while the alternative suggests instability. By analyzing the data and generating a 'KPSS Statistic' and a 'p-value', it helps us grasp whether these prices remain predictable around an average value or if they often fluctuate, impacting their predictability. In simpler terms, it's like a detective checking if the prices of precious metals tend to consistently hover around a typical value or if they frequently stray, making future predictions more challenging.

Transform and try again:

In [ ]:
df_transformed = df.copy()

for f in df.columns:
    df_transformed[f] = df[f].pct_change().fillna(0)

fig = px.line(df_transformed, facet_col="metal", facet_col_wrap=1)
fig.update_yaxes(matches=None)
fig.show()

This code snippet takes a DataFrame 'df' containing columns representing different metals' prices (like gold, platinum, etc.). It creates a transformed DataFrame 'df_transformed' by calculating the percentage change for each metal's price using the `.pct_change()` function. This change computation helps understand how much each metal's price has shifted compared to its previous value. Any missing values resulting from this calculation are filled with zero to maintain continuity. The code then uses the Plotly Express library to generate a line plot with multiple facets (subplots) based on different metals in the DataFrame. Each subplot illustrates the price changes over time for a specific metal, offering a visual comparison of their price movements. Adjustments to the y-axes are made to ensure proper visualization without enforced matching between the different metals' price scales. Ultimately, this plot provides a clear, comparative view of how the prices of various metals fluctuate over time.

In [ ]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('https://github.com/Viral-28/ADS/raw/main/BankChurners.csv')

# Check for missing values
missing_values = df.isnull().any(axis=1)

# Remove the missing value
df = df[~missing_values]

# Save the data
df.to_csv('https://github.com/Viral-28/ADS/raw/main/BankChurners.csv')

df_transformed = df_transformed.dropna()



In [ ]:
df_train, df_test = df_transformed[0:- CFG.tail], df_transformed[-CFG.tail:]

<a id="section-dowhy"></a>
# Causal Inference
Certainly! Causal inference dives into the heart of cause-and-effect relationships within data or systems. When we observe a correlation between two variables, like an increase in one coinciding with changes in another, causal inference seeks to understand if one truly causes the changes in the other or if there's merely an association.

To achieve this, it goes beyond simple observation. It involves techniques and methodologies to approximate causation from observational data, as controlled experiments are often impractical or unethical. This field employs various statistical models, such as regression analysis, propensity scores, or instrumental variables, to tease out causal relationships.

Consider an example where a medication is associated with a health improvement. Causal inference aims to discern if the medication indeed causes the better health or if other factors, like patient lifestyle or genetic makeup, might be responsible for the observed improvement.

It's a challenging area as establishing true causation from observed data requires careful consideration of potential biases, confounding variables, and temporal sequences. Despite these challenges, it's a fundamental field driving insights into understanding the true nature of relationships between different factors, critical in fields like medicine, social sciences, economics, and more.



What steps make up this interface?

1. Constructing a causal mechanism model: Building a causal graph grounded in domain knowledge $\rightarrow$ relying on established information.

2. Determining the target estimand: Formulating an accurate estimand derived from the causal model $\rightarrow$ specifying what needs estimation.

3. Estimating the causal effect: Employing an appropriate method to gauge the effect $\rightarrow$ understanding the method of estimation.

4. Validating estimations: Verifying the estimate's resilience against assumption violations $\rightarrow$ ensuring robustness, akin to how caution can lead to longevity.

<img src="https://blog.ml.cmu.edu/wp-content/uploads/2019/11/structure_diagram-493x1024.jpg">

In [ ]:
!pip install dowhy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.7/377.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.4/174.4 kB 15.6 MB/s eta 0:00:00


<img src="https://getrecast.com/wp-content/uploads/2022/04/Hierarchy-of-Evidence-1.png">

In [ ]:
import dowhy

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df = pd.read_csv('https://github.com/Viral-28/ADS/raw/main/commodity_prices.csv')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df.head()


,date,CRUDE_PETRO,CRUDE_BRENT,CRUDE_DUBAI,CRUDE_WTI,COAL_AUS,COAL_SAFRICA,NGAS_US,NGAS_EUR,NGAS_JP,...,ALUMINUM,IRON_ORE,COPPER,LEAD,Tin,NICKEL,Zinc,GOLD,PLATINUM,SILVER
0,1960M01,1.63,1.63,1.63,..,..,..,0.14,0.4,..,...,511.47,11.42,715.40,206.1,2180.4,1631.0,260.8,35.27,83.5,0.91
1,1960M02,1.63,1.63,1.63,..,..,..,0.14,0.4,..,...,511.47,11.42,728.19,203.7,2180.4,1631.0,244.9,35.27,83.5,0.91
2,1960M03,1.63,1.63,1.63,..,..,..,0.14,0.4,..,...,511.47,11.42,684.94,210.3,2173.8,1631.0,248.7,35.27,83.5,0.91
3,1960M04,1.63,1.63,1.63,..,..,..,0.14,0.4,..,...,511.47,11.42,723.11,213.6,2178.2,1631.0,254.6,35.27,83.5,0.91
4,1960M05,1.63,1.63,1.63,..,..,..,0.14,0.4,..,...,511.47,11.42,684.75,213.4,2162.7,1631.0,253.8,35.27,83.5,0.91


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 72 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           752 non-null    object 
 1   CRUDE_PETRO    752 non-null    float64
 2   CRUDE_BRENT    752 non-null    float64
 3   CRUDE_DUBAI    752 non-null    float64
 4   CRUDE_WTI      752 non-null    object 
 5   COAL_AUS       752 non-null    object 
 6   COAL_SAFRICA   752 non-null    object 
 7   NGAS_US        752 non-null    float64
 8   NGAS_EUR       752 non-null    float64
 9   NGAS_JP        752 non-null    object 
 10  iNATGAS        752 non-null    object 
 11  COCOA          752 non-null    float64
 12  COFFEE_ARABIC  752 non-null    float64
 13  COFFEE_ROBUS   752 non-null    float64
 14  TEA_AVG        752 non-null    float64
 15  TEA_COLOMBO    752 non-null    float64
 16  TEA_KOLKATA    752 non-null    float64
 17  TEA_MOMBASA    752 non-null    float64
 18  COCONUT_OI


The nature of the travel industry, such as whether the booking is handled by a travel agent or tour operator (`market_segment`),
is likely to impact the `lead_time` before the booking.

Additionally, diverse global preferences, including the visitor's `country`, might influence both the `lead_time` (prompting early or late bookings) and preferred `meal` options. Longer `lead_time` could result in a reduced chance of securing a reservation (`days_in_waiting_list`), potentially leading to cancellations (`is_canceled`). Moreover, factors like `days_in_waiting_list`, `total_stay`, and the number of `guests` could influence the decision to cancel or retain a booking.

Similarly, a customer's history of `previous_bookings_not_canceled` may play a role in their cancellation likelihood. While `booking_changes` could affect room assignment and potentially lead to cancellations, it's improbable for this single variable alone to determine the treatment and outcome. Thus, unobserved confounders might encompass other potential causes influencing these scenarios beyond solely `booking_changes`.

In [ ]:
causal_graph = """digraph {
different_room_assigned[label="Different Room Assigned"];
is_canceled[label="Booking Cancelled"];
booking_changes[label="Booking Changes"];
previous_bookings_not_canceled[label="Previous Booking Retentions"];
days_in_waiting_list[label="Days in Waitlist"];
lead_time[label="Lead Time"];
market_segment[label="Market Segment"];
country[label="Country"];
U[label="Unobserved Confounders",observed="no"];
is_repeated_guest;
total_stay;
guests;
meal;
hotel;

U->{different_room_assigned,required_car_parking_spaces,guests,total_stay,total_of_special_requests};

market_segment -> lead_time;
lead_time->is_canceled; country -> lead_time;
different_room_assigned -> is_canceled;
country->meal;
lead_time -> days_in_waiting_list;
days_in_waiting_list ->{is_canceled,different_room_assigned};
previous_bookings_not_canceled -> is_canceled;
previous_bookings_not_canceled -> is_repeated_guest;
is_repeated_guest -> {different_room_assigned,is_canceled};
total_stay -> is_canceled;
guests -> is_canceled;
booking_changes -> different_room_assigned; booking_changes -> is_canceled;
hotel -> {different_room_assigned,is_canceled};
required_car_parking_spaces -> is_canceled;
total_of_special_requests -> {booking_changes,is_canceled};
country->{hotel, required_car_parking_spaces,total_of_special_requests};
market_segment->{hotel, required_car_parking_spaces,total_of_special_requests};
}"""

* Treatment involves allocating the identical room type reserved by the customer during the booking process.
* Outcome refers to whether the booking was ultimately canceled or retained.
* Common Causes encompass the variables that, in our perspective, exert a causal influence on both the Outcome and Treatment. According to our causal assumptions, the two variables meeting this criterion are `booking_changes` and other Unobserved Confounders.
* To facilitate the determination of the causal effect, **we eliminate the unobserved confounder node from the graphical representation**.


In [ ]:
df=df.replace('-', np.nan)

In [ ]:
df=df.replace('\n4', '',regex=True)

In [ ]:
# Random Common Cause:- Adds randomly drawn covariates to data and re-runs the analysis to see if the causal estimate changes or not.
# If our assumption was originally correct then the causal estimate shouldn't change by much.

refute1_results = model.refute_estimate(identified_estimand, estimate,
                                      method_name="random_common_cause")
print(refute1_results)

Refute: Add a random common cause
Estimated effect:-0.26208282041490744
New effect:-0.26208282041490744
p value:1.0



In [ ]:
# Placebo Treatment Refuter:- Randomly assigns any covariate as a treatment and re-runs the analysis. If our assumptions were correct then this newly found out estimate should go to 0.

refute2_results = model.refute_estimate(identified_estimand, estimate,
        method_name="placebo_treatment_refuter")
print(refute2_results)



Refute: Use a Placebo Treatment
Estimated effect:-0.26208282041490744
New effect:0.055730578659796484
p value:0.0



In [ ]:
# Data Subset Refuter:- Creates subsets of the data(similar to cross-validation) and checks whether the causal estimates vary across subsets.
# If our assumptions were correct there shouldn't be much variation.

refute3_results = model.refute_estimate(identified_estimand, estimate,
        method_name="data_subset_refuter")
print(refute3_results)


Refute: Use a subset of data
Estimated effect:-0.26208282041490744
New effect:-0.26209624289679495
p value:0.88



The "Data Subset Refuter" method functions akin to cross-validation by generating various subsets of the data and examining whether the causal estimates differ across these subsets. The premise is that if our causal assumptions are accurate, there should be minimal variation in the estimates across different subsets.

Upon execution, the `refute_estimate` function analyzes the identified estimand and the estimated causal effect using this specific method. The `refute3_results` variable stores the results obtained from this process. These results can offer insights into the stability and consistency of the causal estimates when the data is partitioned into subsets, helping to evaluate the robustness of the causal inference.

#Conclusion
Granger causality and causal inference offer valuable insights into relationships between variables, shedding light on potential cause-and-effect dynamics. Here are some conclusions you might draw based on their application:

1. **Understanding Interconnectedness:** Granger causality analysis provides a framework to assess temporal relationships between variables. By determining if one time series provides predictive information about another, it helps identify potential interdependencies, showcasing how variables might influence each other over time.

2. **Identifying Potential Causation:** Causal inference, on the other hand, aims to discern true causal relationships from observational data. By employing various statistical methods and considering temporal order, it helps unveil potential cause-and-effect connections between variables, going beyond mere correlation.

3. **Decision-Making Insights:** Together, these methodologies can offer critical insights for decision-making. Granger causality provides predictive leads, while causal inference helps establish the more robust and potentially causal relationships among variables. These insights can aid in making informed decisions, especially in fields like economics, social sciences, healthcare, and beyond.

4. **Robustness Testing:** Applying multiple robustness checks, as seen in causal inference, enhances confidence in identified relationships. It's crucial to subject these relationships to various tests to ensure stability and validity across different scenarios or subsets of data, validating the reliability of the inferred causal connections.

5. **Cautious Interpretation:** While both methodologies provide valuable insights, it's important to interpret their results carefully. Granger causality might highlight temporal correlations, but it doesn't imply causation. Causal inference strives to establish causation, yet it's crucial to consider potential biases and unobserved confounders.

In conclusion, integrating insights from Granger causality and causal inference can provide a more comprehensive understanding of relationships within datasets, aiding in decision-making and offering deeper insights into potential cause-and-effect dynamics.

#References
Medium Article https://towardsdatascience.com/inferring-causality-in-time-series-data-b8b75fe52c46

Science Direct https://www.sciencedirect.com/topics/social-sciences/causal-inference#:~:text=Causal%20inference%20refers%20to%20an,causal%20conclusions%20based%20on%20data.

Harvard article https://www.hsph.harvard.edu/miguel-hernan/causal-inference-book/

Aptech https://www.aptech.com/blog/introduction-to-granger-causality/

Medium article https://towardsdatascience.com/a-quick-introduction-on-granger-causality-testing-for-time-series-analysis-7113dc9420d2


MIT License

Copyright (c) 2023 Viral Jangale

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

